In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('SparkWindowFunctionsExamples').getOrCreate()

In [4]:
# Create sample dataframe

simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [5]:
df.createOrReplaceTempView("employees")

In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, row_number, rank, dense_rank, lag, lead, col, avg, sum, min, max 

**row_number Window Function**

row_number() window function is used to give the sequential row number starting from 1 to the result of each window partition.

In [7]:
# Partition by Department and ascending salary.

df.withColumn("row_number",row_number().over(Window.partitionBy("department").orderBy("salary"))) \
    .show()

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         2|
|       Robert|     Sales|  4100|         3|
|         Saif|     Sales|  4100|         4|
|      Michael|     Sales|  4600|         5|
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
+-------------+----------+------+----------+



In [8]:
spark.sql("SELECT employee_name, department, salary, \
          row_number() over (partition by department order by salary) row_number \
          FROM employees").show()

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         2|
|       Robert|     Sales|  4100|         3|
|         Saif|     Sales|  4100|         4|
|      Michael|     Sales|  4600|         5|
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
+-------------+----------+------+----------+



In [9]:
# Partition by Department and descending salary.

df.withColumn("row_number",row_number().over(Window.partitionBy("department").orderBy(desc("salary")))) \
    .show()

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|      Michael|     Sales|  4600|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         3|
|        James|     Sales|  3000|         4|
|        James|     Sales|  3000|         5|
|          Jen|   Finance|  3900|         1|
|        Scott|   Finance|  3300|         2|
|        Maria|   Finance|  3000|         3|
|         Jeff| Marketing|  3000|         1|
|        Kumar| Marketing|  2000|         2|
+-------------+----------+------+----------+



In [10]:
spark.sql("SELECT employee_name, department, salary, \
          row_number() over (partition by department order by salary desc) row_number \
          FROM employees").show()

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|      Michael|     Sales|  4600|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         3|
|        James|     Sales|  3000|         4|
|        James|     Sales|  3000|         5|
|          Jen|   Finance|  3900|         1|
|        Scott|   Finance|  3300|         2|
|        Maria|   Finance|  3000|         3|
|         Jeff| Marketing|  3000|         1|
|        Kumar| Marketing|  2000|         2|
+-------------+----------+------+----------+



**rank Window Function**

rank() window function is used to provide a rank to the result within a window partition. This function leaves gaps in rank when there are ties.

In [11]:
# Partition by Department and ascending salary.

df.withColumn("rank",rank().over(Window.partitionBy("department").orderBy("salary"))).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|        James|     Sales|  3000|   1|
|        James|     Sales|  3000|   1|
|       Robert|     Sales|  4100|   3|
|         Saif|     Sales|  4100|   3|
|      Michael|     Sales|  4600|   5|
|        Maria|   Finance|  3000|   1|
|        Scott|   Finance|  3300|   2|
|          Jen|   Finance|  3900|   3|
|        Kumar| Marketing|  2000|   1|
|         Jeff| Marketing|  3000|   2|
+-------------+----------+------+----+



In [12]:
spark.sql("SELECT employee_name, department, salary, rank() over(partition by department order by salary) rank \
            FROM employees")\
            .show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|        James|     Sales|  3000|   1|
|        James|     Sales|  3000|   1|
|       Robert|     Sales|  4100|   3|
|         Saif|     Sales|  4100|   3|
|      Michael|     Sales|  4600|   5|
|        Maria|   Finance|  3000|   1|
|        Scott|   Finance|  3300|   2|
|          Jen|   Finance|  3900|   3|
|        Kumar| Marketing|  2000|   1|
|         Jeff| Marketing|  3000|   2|
+-------------+----------+------+----+



In [13]:
# Partition by Department and descending salary.

df.withColumn("rank",rank().over(Window.partitionBy("department").orderBy(desc("salary")))).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|   1|
|       Robert|     Sales|  4100|   2|
|         Saif|     Sales|  4100|   2|
|        James|     Sales|  3000|   4|
|        James|     Sales|  3000|   4|
|          Jen|   Finance|  3900|   1|
|        Scott|   Finance|  3300|   2|
|        Maria|   Finance|  3000|   3|
|         Jeff| Marketing|  3000|   1|
|        Kumar| Marketing|  2000|   2|
+-------------+----------+------+----+



In [14]:
spark.sql("SELECT employee_name, department, salary, rank() over(partition by department order by salary desc) rank \
            FROM employees")\
            .show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|   1|
|       Robert|     Sales|  4100|   2|
|         Saif|     Sales|  4100|   2|
|        James|     Sales|  3000|   4|
|        James|     Sales|  3000|   4|
|          Jen|   Finance|  3900|   1|
|        Scott|   Finance|  3300|   2|
|        Maria|   Finance|  3000|   3|
|         Jeff| Marketing|  3000|   1|
|        Kumar| Marketing|  2000|   2|
+-------------+----------+------+----+



**dense_rank Window**

dense_rank() window function is used to get the result with rank of rows within a window partition without any gaps. This is similar to rank() function difference being rank function leaves gaps in rank when there are ties.

In [15]:
# Partition by Department and ascending salary.

df.withColumn("rank",dense_rank().over(Window.partitionBy("department").orderBy("salary"))).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|        James|     Sales|  3000|   1|
|        James|     Sales|  3000|   1|
|       Robert|     Sales|  4100|   2|
|         Saif|     Sales|  4100|   2|
|      Michael|     Sales|  4600|   3|
|        Maria|   Finance|  3000|   1|
|        Scott|   Finance|  3300|   2|
|          Jen|   Finance|  3900|   3|
|        Kumar| Marketing|  2000|   1|
|         Jeff| Marketing|  3000|   2|
+-------------+----------+------+----+



In [16]:
spark.sql("SELECT employee_name, department, salary, dense_rank() over(partition by department order by salary) dense_rank \
          FROM employees") \
          .show()

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         2|
|      Michael|     Sales|  4600|         3|
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
+-------------+----------+------+----------+



In [17]:
# Partition by Department and descending salary.

df.withColumn("rank",dense_rank().over(Window.partitionBy("department").orderBy(desc("salary")))).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|   1|
|       Robert|     Sales|  4100|   2|
|         Saif|     Sales|  4100|   2|
|        James|     Sales|  3000|   3|
|        James|     Sales|  3000|   3|
|          Jen|   Finance|  3900|   1|
|        Scott|   Finance|  3300|   2|
|        Maria|   Finance|  3000|   3|
|         Jeff| Marketing|  3000|   1|
|        Kumar| Marketing|  2000|   2|
+-------------+----------+------+----+



In [18]:
spark.sql("SELECT employee_name, department, salary, dense_rank() over(partition by department order by salary desc) dense_rank \
          FROM employees") \
          .show()

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|      Michael|     Sales|  4600|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         2|
|        James|     Sales|  3000|         3|
|        James|     Sales|  3000|         3|
|          Jen|   Finance|  3900|         1|
|        Scott|   Finance|  3300|         2|
|        Maria|   Finance|  3000|         3|
|         Jeff| Marketing|  3000|         1|
|        Kumar| Marketing|  2000|         2|
+-------------+----------+------+----------+



**lag Window Function**

This is the same as the LAG function in SQL.

In [19]:
# Partition by Department and ascending salary.

df.withColumn("lag",lag("salary").over(Window.partitionBy("department").orderBy("salary"))).show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|        James|     Sales|  3000|null|
|        James|     Sales|  3000|3000|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|4100|
|      Michael|     Sales|  4600|4100|
|        Maria|   Finance|  3000|null|
|        Scott|   Finance|  3300|3000|
|          Jen|   Finance|  3900|3300|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|2000|
+-------------+----------+------+----+



In [20]:
spark.sql("SELECT employee_name, department, salary, lag(salary) over(partition by department order by salary) lag \
            FROM employees") \
            .show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|        James|     Sales|  3000|null|
|        James|     Sales|  3000|3000|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|4100|
|      Michael|     Sales|  4600|4100|
|        Maria|   Finance|  3000|null|
|        Scott|   Finance|  3300|3000|
|          Jen|   Finance|  3900|3300|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|2000|
+-------------+----------+------+----+



In [21]:
df.withColumn("lag",lag("salary",2,0).over(Window.partitionBy("department").orderBy("salary"))).show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|        James|     Sales|  3000|   0|
|        James|     Sales|  3000|   0|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|3000|
|      Michael|     Sales|  4600|4100|
|        Maria|   Finance|  3000|   0|
|        Scott|   Finance|  3300|   0|
|          Jen|   Finance|  3900|3000|
|        Kumar| Marketing|  2000|   0|
|         Jeff| Marketing|  3000|   0|
+-------------+----------+------+----+



In [22]:
spark.sql("SELECT employee_name, department, salary, lag(salary,2,0) over(partition by department order by salary) lag \
            FROM employees") \
            .show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|        James|     Sales|  3000|   0|
|        James|     Sales|  3000|   0|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|3000|
|      Michael|     Sales|  4600|4100|
|        Maria|   Finance|  3000|   0|
|        Scott|   Finance|  3300|   0|
|          Jen|   Finance|  3900|3000|
|        Kumar| Marketing|  2000|   0|
|         Jeff| Marketing|  3000|   0|
+-------------+----------+------+----+



In [23]:
# Partition by Department and descending salary.

df.withColumn("lag",lag("salary").over(Window.partitionBy("department").orderBy(desc("salary")))).show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|null|
|       Robert|     Sales|  4100|4600|
|         Saif|     Sales|  4100|4100|
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|3000|
|          Jen|   Finance|  3900|null|
|        Scott|   Finance|  3300|3900|
|        Maria|   Finance|  3000|3300|
|         Jeff| Marketing|  3000|null|
|        Kumar| Marketing|  2000|3000|
+-------------+----------+------+----+



In [24]:
spark.sql("SELECT employee_name, department, salary, lag(salary) over(partition by department order by salary desc) lag \
            FROM employees") \
            .show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|null|
|       Robert|     Sales|  4100|4600|
|         Saif|     Sales|  4100|4100|
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|3000|
|          Jen|   Finance|  3900|null|
|        Scott|   Finance|  3300|3900|
|        Maria|   Finance|  3000|3300|
|         Jeff| Marketing|  3000|null|
|        Kumar| Marketing|  2000|3000|
+-------------+----------+------+----+



In [25]:
df.withColumn("lag",lag("salary",2,0).over(Window.partitionBy("department").orderBy(desc("salary")))).show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|   0|
|       Robert|     Sales|  4100|   0|
|         Saif|     Sales|  4100|4600|
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|4100|
|          Jen|   Finance|  3900|   0|
|        Scott|   Finance|  3300|   0|
|        Maria|   Finance|  3000|3900|
|         Jeff| Marketing|  3000|   0|
|        Kumar| Marketing|  2000|   0|
+-------------+----------+------+----+



In [26]:
spark.sql("SELECT employee_name, department, salary, lag(salary,2,0) over(partition by department order by salary desc) lag \
            FROM employees") \
            .show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|   0|
|       Robert|     Sales|  4100|   0|
|         Saif|     Sales|  4100|4600|
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|4100|
|          Jen|   Finance|  3900|   0|
|        Scott|   Finance|  3300|   0|
|        Maria|   Finance|  3000|3900|
|         Jeff| Marketing|  3000|   0|
|        Kumar| Marketing|  2000|   0|
+-------------+----------+------+----+



**lead Window Function**

This is the same as the LEAD function in SQL.

In [27]:
# Partition by Department and ascending salary.

df.withColumn("lead",lead("salary").over(Window.partitionBy("department").orderBy("salary"))).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        James|     Sales|  3000|3000|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4100|
|         Saif|     Sales|  4100|4600|
|      Michael|     Sales|  4600|null|
|        Maria|   Finance|  3000|3300|
|        Scott|   Finance|  3300|3900|
|          Jen|   Finance|  3900|null|
|        Kumar| Marketing|  2000|3000|
|         Jeff| Marketing|  3000|null|
+-------------+----------+------+----+



In [28]:
spark.sql("SELECT employee_name, department, salary, lead(salary) over(partition by department order by salary) lead \
          FROM employees") \
          .show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        James|     Sales|  3000|3000|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4100|
|         Saif|     Sales|  4100|4600|
|      Michael|     Sales|  4600|null|
|        Maria|   Finance|  3000|3300|
|        Scott|   Finance|  3300|3900|
|          Jen|   Finance|  3900|null|
|        Kumar| Marketing|  2000|3000|
|         Jeff| Marketing|  3000|null|
+-------------+----------+------+----+



In [29]:
df.withColumn("lead",lead("salary",2,0).over(Window.partitionBy("department").orderBy("salary"))).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4600|
|         Saif|     Sales|  4100|   0|
|      Michael|     Sales|  4600|   0|
|        Maria|   Finance|  3000|3900|
|        Scott|   Finance|  3300|   0|
|          Jen|   Finance|  3900|   0|
|        Kumar| Marketing|  2000|   0|
|         Jeff| Marketing|  3000|   0|
+-------------+----------+------+----+



In [30]:
spark.sql("SELECT employee_name, department, salary, lead(salary,2,0) over(partition by department order by salary) lead \
          FROM employees") \
          .show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4600|
|         Saif|     Sales|  4100|   0|
|      Michael|     Sales|  4600|   0|
|        Maria|   Finance|  3000|3900|
|        Scott|   Finance|  3300|   0|
|          Jen|   Finance|  3900|   0|
|        Kumar| Marketing|  2000|   0|
|         Jeff| Marketing|  3000|   0|
+-------------+----------+------+----+



In [31]:
# Partition by Department and descending salary.

df.withColumn("lead",lead("salary").over(Window.partitionBy("department").orderBy(desc("salary")))).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|4100|
|       Robert|     Sales|  4100|4100|
|         Saif|     Sales|  4100|3000|
|        James|     Sales|  3000|3000|
|        James|     Sales|  3000|null|
|          Jen|   Finance|  3900|3300|
|        Scott|   Finance|  3300|3000|
|        Maria|   Finance|  3000|null|
|         Jeff| Marketing|  3000|2000|
|        Kumar| Marketing|  2000|null|
+-------------+----------+------+----+



In [32]:
spark.sql("SELECT employee_name, department, salary, lead(salary) over(partition by department order by salary desc) lead \
          FROM employees") \
          .show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|4100|
|       Robert|     Sales|  4100|4100|
|         Saif|     Sales|  4100|3000|
|        James|     Sales|  3000|3000|
|        James|     Sales|  3000|null|
|          Jen|   Finance|  3900|3300|
|        Scott|   Finance|  3300|3000|
|        Maria|   Finance|  3000|null|
|         Jeff| Marketing|  3000|2000|
|        Kumar| Marketing|  2000|null|
+-------------+----------+------+----+



In [33]:
df.withColumn("lead",lead("salary",2,0).over(Window.partitionBy("department").orderBy(desc("salary")))).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|4100|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|3000|
|        James|     Sales|  3000|   0|
|        James|     Sales|  3000|   0|
|          Jen|   Finance|  3900|3000|
|        Scott|   Finance|  3300|   0|
|        Maria|   Finance|  3000|   0|
|         Jeff| Marketing|  3000|   0|
|        Kumar| Marketing|  2000|   0|
+-------------+----------+------+----+



In [34]:
spark.sql("SELECT employee_name, department, salary, lead(salary,2,0) over(partition by department order by salary desc) lead \
          FROM employees") \
          .show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|      Michael|     Sales|  4600|4100|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|3000|
|        James|     Sales|  3000|   0|
|        James|     Sales|  3000|   0|
|          Jen|   Finance|  3900|3000|
|        Scott|   Finance|  3300|   0|
|        Maria|   Finance|  3000|   0|
|         Jeff| Marketing|  3000|   0|
|        Kumar| Marketing|  2000|   0|
+-------------+----------+------+----+



**PySpark Window Aggregate Functions**

In this section, I will explain how to calculate sum, min, max for each department using PySpark SQL Aggregate window functions and WindowSpec. When working with Aggregate functions, we don’t need to use order by clause.

In [35]:
df.withColumn("row",row_number().over(Window.partitionBy("department").orderBy("salary")))\
  .withColumn("max_dept_sal",max("salary").over(Window.partitionBy("department")))\
  .withColumn("min_dept_sal",min("salary").over(Window.partitionBy("department")))\
  .withColumn("avg_dept_sal",avg("salary").over(Window.partitionBy("department")))\
  .withColumn("sum_dept_sal",sum("salary").over(Window.partitionBy("department")))\
  .where(col("row")==1)\
  .select("department", "avg_dept_sal", "sum_dept_sal", "min_dept_sal", "max_dept_sal")\
  .show()

+----------+------------+------------+------------+------------+
|department|avg_dept_sal|sum_dept_sal|min_dept_sal|max_dept_sal|
+----------+------------+------------+------------+------------+
|     Sales|      3760.0|       18800|        3000|        4600|
|   Finance|      3400.0|       10200|        3000|        3900|
| Marketing|      2500.0|        5000|        2000|        3000|
+----------+------------+------------+------------+------------+



In [36]:
spark.sql("SELECT department, avg(salary) as avg_dept_sal \
          , sum(salary) as sum_dept_sal \
          , min(salary) as min_dept_sal \
          , max(salary) as max_dept_sal \
          FROM employees \
          GROUP BY department \
          ") \
        .show()

+----------+------------+------------+------------+------------+
|department|avg_dept_sal|sum_dept_sal|min_dept_sal|max_dept_sal|
+----------+------------+------------+------------+------------+
|     Sales|      3760.0|       18800|        3000|        4600|
|   Finance|      3400.0|       10200|        3000|        3900|
| Marketing|      2500.0|        5000|        2000|        3000|
+----------+------------+------------+------------+------------+

